In [ ]:
from imutils.perspective import four_point_transform
from imutils import contours
import imutils
import cv2
import numpy as np
import argparse
import random
import matplotlib.pyplot as plt

def show_images(images, titles, kill_later=True):
    for index, image in enumerate(images):
        cv2.imshow(titles[index], image)
    cv2.waitKey(0)
    if kill_later:
        cv2.destroyAllWindows()
        
# edge detection
image = cv2.imread( "images/scan_test.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 75, 200)
show_images([edged], ["Edged"])

# find contours in edge detected image
cnts = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
docCnt = None

allContourImage = image.copy()
cv2.drawContours(allContourImage, cnts, -1, (0, 0, 255), 3)
print("Total contours found after edge detection {}".format(len(cnts)))
show_images([allContourImage], ["All contours from edge detected image"])

# finding the document contour
if len(cnts) > 0:
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    for c in cnts:
        peri = cv2.arcLength(c, closed=True)
        approx = cv2.approxPolyDP(c, epsilon=peri*0.02, closed=True)

        if len(approx) == 4:
            docCnt = approx
            break

contourImage = image.copy()
cv2.drawContours(contourImage, [docCnt], -1, (0, 0, 255), 2)
show_images([contourImage], ["Outline"])

# Getting the bird's eye view, top-view of the document
paper = four_point_transform(image, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))
show_images([paper, warped], ["Paper", "Gray"])